### Importação do pacote para uso de cubos do Microstrategy

In [1]:
from mstrio import microstrategy

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
#from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy

In [3]:
conn = microstrategy.Connection(base_url="http://aesc-biweb-01.hmd.local:8080/MicroStrategyLibrary/api", username="administrator", password="ferias2018", project_name="IA")
conn.connect()

### Leitura dos Datasets

In [4]:
agenda2 = conn.get_report(report_id='FECD8AEF4DA2760D0342AD8123C26C25')
produtos2 = conn.get_report(report_id='988CD01740D0F6FE7A9244AFD07B765C')
procedimentos2 = conn.get_report(report_id='ABC22A9346139589A76381B525C7A377')

### Preciso exportar para conseguir que os objetos ficam com o mesmo datatype

In [5]:
agenda2.to_excel('obs_2019.xlsx')
agenda2.to_excel('prod_2019.xlsx')
agenda2.to_excel('proc_2019.xlsx')

### Leitura do excel recem exportado.

In [6]:
cut_num = 60000
agenda = pd.read_excel('obs_2019.xlsx', nrows=cut_num)
produtos = pd.read_excel('prod_2019.xlsx') 
procedimentos = pd.read_excel('proc_2019.xlsx')

In [7]:
agenda = agenda.astype({'CD_AVISO_CIRURGIA':int})
#agenda2 = agenda2.astype({'CD_AVISO_CIRURGIA':int})

In [11]:
print (agenda)

       Unnamed: 0  CD_AVISO_CIRURGIA  CD_CIR  PROCEDIMENTO  VL_IDADE  \
0               0             502273     NaN           NaN      53.0   
1               1             512641     NaN           NaN      17.0   
2               2             512641     NaN           NaN      17.0   
3               3             514990     NaN           NaN      38.0   
4               4             514990     NaN           NaN      38.0   
...           ...                ...     ...           ...       ...   
33761         761             640593     NaN           NaN      65.0   
33762         762             640612     NaN           NaN      76.0   
33763         763             640629     NaN           NaN      70.0   
33764         764             640783     NaN           NaN      73.0   
33765         765             640783     NaN           NaN      73.0   

                                           DS_OBSERVACAO  DS_PRODUTO  \
0      BX DE VÉSPERA    COM UTI AG POR SEC MICHELLE D...       

### Junção dos datasets.;

In [8]:
import sys
for idx in agenda.index:
    cd_aviso = agenda.loc[idx, 'CD_AVISO_CIRURGIA']
    prods = produtos[produtos['CD_AVISO_CIRURGIA'] == cd_aviso]
    procs = procedimentos[procedimentos['CD_AVISO_CIRURGIA'] == cd_aviso]
    
    if pd.isnull(agenda.loc[idx, 'DS_OBSERVACAO']):
        agenda.loc[idx, 'DS_OBSERVACAO'] = ''
    if len(prods) > 0:
        agenda.loc[idx, 'DS_OBSERVACAO'] += ' ' + (' ').join(prods['DS_PRODUTO'].astype('str').values)
    if len(procs) > 0:
        agenda.loc[idx, 'DS_OBSERVACAO'] += ' ' + (' ').join(procs['PROCEDIMENTO'].astype('str').values)
    if (idx % 100) == 0:
        sys.stdout.write(str(idx) + ', ')
        sys.stdout.flush()
    if (idx % 2000) == 0:
        print('')

  

0, 
100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 
2100, 2200, 2300, 2400, 2500, 2600, 2700, 2800, 2900, 3000, 3100, 3200, 3300, 3400, 3500, 3600, 3700, 3800, 3900, 4000, 
4100, 4200, 4300, 4400, 4500, 4600, 4700, 4800, 4900, 5000, 5100, 5200, 5300, 5400, 5500, 5600, 5700, 5800, 5900, 6000, 
6100, 6200, 6300, 6400, 6500, 6600, 6700, 6800, 6900, 7000, 7100, 7200, 7300, 7400, 7500, 7600, 7700, 7800, 7900, 8000, 
8100, 8200, 8300, 8400, 8500, 8600, 8700, 8800, 8900, 9000, 9100, 9200, 9300, 9400, 9500, 9600, 9700, 9800, 9900, 10000, 
10100, 10200, 10300, 10400, 10500, 10600, 10700, 10800, 10900, 11000, 11100, 11200, 11300, 11400, 11500, 11600, 11700, 11800, 11900, 12000, 
12100, 12200, 12300, 12400, 12500, 12600, 12700, 12800, 12900, 13000, 13100, 13200, 13300, 13400, 13500, 13600, 13700, 13800, 13900, 14000, 
14100, 14200, 14300, 14400, 14500, 14600, 14700, 14800, 14900, 15000, 15100, 15200, 15300, 15400, 15500, 15600, 15700

### Configuração do dataset

In [9]:
import numpy as np
num_features = 500
vectorizer = TfidfVectorizer(max_features=num_features).fit(agenda['DS_OBSERVACAO'].values)
data = vectorizer.transform(agenda['DS_OBSERVACAO'].values)
data = data.todense()
## adiciona idade
mean_idade = int(np.mean(agenda['VL_IDADE']))
values = {'VL_IDADE': mean_idade}
agenda.fillna(value=values, inplace=True)
idades = agenda['VL_IDADE'].astype('int').values.reshape(data.shape[0],1)
data = np.append(data, idades, axis=1)
data.shape

(33766, 501)

In [10]:
print(agenda['SN_UTI'])

0        1
1        1
2        1
3        0
4        0
        ..
33761    0
33762    1
33763    0
33764    0
33765    0
Name: SN_UTI, Length: 33766, dtype: int64


### Criação dos dados de treino e teste e aplicação de RandomForest

In [52]:
scores = cross_val_score(RandomForestClassifier(n_estimators=50), data, agenda['SN_UTI'])
classifier = RandomForestClassifier(n_estimators=50).fit(data, agenda['SN_UTI'].values)
scores, np.mean(scores)



C:\Users\maicon.freitas\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


(array([0.89472281, 0.90075522, 0.90031097]), 0.8985963357673775)

In [53]:
features = {}

### Determina o peso das features

In [54]:
import operator
features = {}
ridx = {idx:term for term,idx in vectorizer.vocabulary_.items()}
for i,v in enumerate(classifier.feature_importances_):
    if i >= num_features:
        features['IDADE'] = v
        break
        
    if not ridx[i] in features.keys():
        features[ridx[i]] = v
    else:
        features[ridx[i]] += v
        if (i % 100) == 0:
            sys.stdout.write(str(i) + ', ')
            sys.stdout.flush()


In [55]:
sorted(features.items(), key=operator.itemgetter(1), reverse=True)[:20]

[('cti', 0.09922276432226035),
 ('IDADE', 0.08139431922806015),
 ('internado', 0.03857282174070416),
 ('paciente', 0.026479998965129436),
 ('nan', 0.024545694855751105),
 ('dr', 0.01741026117654804),
 ('dn', 0.0164451106235299),
 ('enf', 0.016153263453143166),
 ('por', 0.01537713404354683),
 ('no', 0.0147820768417614),
 ('na', 0.014503949347724018),
 ('ag', 0.014156173854496563),
 ('amb', 0.013941112851564012),
 ('com', 0.013384434045754628),
 ('mat', 0.013105547430219345),
 ('docs', 0.012790214204260542),
 ('internada', 0.011914375185346389),
 ('pos', 0.011248110017107103),
 ('leito', 0.010285496533066684),
 ('bx', 0.010100434201882058)]

### Exporta o modelo

In [56]:
import pickle
pickle.dump(classifier, open('agenda_classifier.pckls', 'wb'))
pickle.dump(vectorizer, open('agenda_vectorizer.pckls', 'wb'))
agenda.to_csv('agenda.csv.gz', compression='gzip', index=None)
#from google.colab import files
#files.download('agenda.csv.gz')

### Aplica o modelo

In [68]:
agendaTeste = pd.read_excel('obs_2019.xlsx', nrows=5000, skiprows=cut_num, header=0)

agendaTeste.columns = agenda.columns

for idx in agendaTeste.index:
    cd_aviso = agendaTeste.loc[idx, 'CD_AVISO_CIRURGIA']
    prods = produtos[produtos['CD_AVISO_CIRURGIA'] == cd_aviso]
    procs = procedimentos[procedimentos['CD_AVISO_CIRURGIA'] == cd_aviso]
   
    #if len(prods) > 0:

        #agendaTeste.loc[idx, 'DS_OBSERVACAO'] += ' ' + (' ').join(prods['DS_PRODUTO'].astype)

    #if len(procs) > 0:
        #agendaTeste.loc[idx, 'DS_OBSERVACAO'] += ' ' + (' ').join(procs['PROCEDIMENTO'].astype)
        
    if (idx % 100) == 0:
        sys.stdout.write(str(idx) + ', ')
        sys.stdout.flush()


ValueError: Length mismatch: Expected axis has 0 elements, new values have 11 elements

In [63]:
data_teste = vectorizer.transform(agendaTeste['DS_OBSERVACAO'].astype('str').values)
data_teste = data_teste.todense()

In [64]:
mean_idade = int(np.mean(agendaTeste['VL_IDADE']))
values = {'VL_IDADE': mean_idade}
agendaTeste.fillna(value=values, inplace=True)
idades = agendaTeste['VL_IDADE'].astype('int').values.reshape(data_teste.shape[0],1)

data_teste = np.append(data_teste, idades, axis=1)
data_teste.shape

(5000, 501)

In [65]:
agendaTeste['pred'] = classifier.predict(data_teste)

### Resultados

In [66]:
from sklearn.metrics import classification_report
print(classification_report(agendaTeste['SN_UTI'], agendaTeste['pred'].values))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      4449
           1       0.98      0.59      0.74       551

    accuracy                           0.95      5000
   macro avg       0.97      0.79      0.86      5000
weighted avg       0.95      0.95      0.95      5000



### Publica Resultados

In [ ]:
novo_cubo=1

if novo_cubo == 1:
    newDatasetIdmes, newTableIdmes = conn.create_dataset(data_frame= agendaTeste, dataset_name='Cubo_Agendamentos', table_name='Agenda')
else:
    conn.update_dataset(data_frame=agendaTeste, dataset_id='6EED37184F7DD1E192C43C9CDCF78AC9', table_name='Agendamentos', update_policy='replace')


